 # Module

In [1]:
from dotenv import load_dotenv
import os
from pydantic import BaseModel
import functools
import operator
import json
from IPython.display import Image, display

from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, AIMessage, ToolMessage, HumanMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_experimental.tools import PythonREPLTool
from langgraph.prebuilt import create_react_agent

# .env 파일 로드
load_dotenv()

True

# Create Tools

In [2]:
# search engine
tavily_tool = TavilySearchResults(max_results=5)

# code executer for chart
python_repl_tool = PythonREPLTool()

# Helper Utility
* Agent의 응답을 Human Message로 변환하는 역할

In [3]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {
        "messages": [HumanMessage(content=result["messages"][-1].content, name=name)]
    }

# Create Agent Supervisor
* 관리자 Agent는 필요에 따라 fuction calling을 출력할 것이며, 필요하지 않을 때는 finish를 출력하는 역할을 담당한다.

In [4]:
# 사용할 수 있는 fuctions
members = ["Researcher", "Coder", "Writer"]

# # supervisor의 system 프롬프트
# system_prompt = (
#     "You are a supervisor tasked with managing a conversation between the"
#     " following workers:  {members}. Given the following user request,"
#     " respond with the worker to act next. Each worker will perform a"
#     " task and respond with their results and status. When finished,"
#     " respond with FINISH."
# )

system_prompt = (
    "당신은 고객이 보내준 연구 내용을 보고, 연구노트를 작성해 주는 에이전시의 관리자입니다."
    "에이전시는 연구노트 작성을 위해 다양한 작업자와 협업을 진행합니다."
    "작업자들 : {members}"
    "Researcher 작업자는 연구 노트의 내용을 더 풍부하게 하기 위해, 필요한 논문이나 자료를 검색해주는 역할을 합니다. Researcher 작업자는 꼭 한번 이상 호출을 하세요."
    "Coder 작업자는 연구 내용에 특정 수치가 있거나 그래프가 필요한 경우, python 코드를 동작시켜 시각화를 해주는 역할을 합니다. 시각화가 필요없는 경우 호출하지 않아도 됩니다."
    "Writer 작업자는 각각의 작업자들이 수행한 결과를 최종적으로 종합하여 연구노트 형식에 맞게 작성해 주는 역할을 합니다. Researcher와 Coder 작업자들이 모두 작업을 완료하면 호출하시면 됩니다."
    "각각의 작업자들은 자신의 작업을 수행하고, 결과와 상태를 반환합니다."
    "고객이 연구 내용을 보내 주면, 필요한 작업자를 선택하여 작업자를 반환해 주십시오."
    "마지막에는 항상 Writer 작업자를 호출하시면 되며, Writer 작업자가 연구노트 작성을 완료하면 FINISH를 선택하여 종료합니다."
)

# Supervisor가 출력할 수 있는 options
options = ["FINISH"] + members

# Supervisor의 출력 스키마 정의
class routeResponse(BaseModel):
    next: Literal[*options]

# Supervisor의 전체 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "위 대화를 고려할 때, 다음 작업을 진행해야할 작업자는 누구인가요?"
            "아니면 FINISH 해야할까요? 다음 중 하나를 선택해 주세요 : {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

# LLM 정의
llm = ChatOpenAI(model="gpt-4o")

# Supervisor 노드 정의
def supervisor_agent(state):
    supervisor_chain = prompt | llm.with_structured_output(routeResponse)
    return supervisor_chain.invoke(state)

# Graph 설계

* State 정의

In [5]:
# State 정의
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str

* Research Node 정의

In [ ]:
research_system_prompt = (
    "당신은 고객이 보내준 연구 내용을 보고, 연구 노트에서 더 조사할 만한 요소를 조사해 주는 작업자 입니다."
    "연구 내용과 관련하여 논문이나 자료를 검색하여, 연구 노트에 추가할 수 있는 정보를 찾아주세요."
    ""
)

research_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", research_system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "위 대화에서, 연구 내용과 관련하여 논문이나 자료를 검색하여, 연구 노트에 추가할 수 있는 정보를 찾아주세요."
        ),
    ]
)
research_agent = create_react_agent(llm, tools=[tavily_tool],research_prompt)
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

TypeError: create_react_agent() takes 2 positional arguments but 3 were given

* Coder Node 정의

In [ ]:
coder_system_prompt = (
    "당신은 고객이 보내준 연구 내용을 보고, 연구 노트에서 시각화 할 수 있는 모든 것들을 시각화해 주는 작업자 입니다."
    "연구 내용에 기재된 수치나 그래프 등 시각화가 가능한 요소를 찾아, python 코드를 동작시켜 시각화를 해주세요."
    "최종적으로 시각화 한 사진을 다운로드 받을 수 있는 링크를 전달 해 주세요"
    ""
)

coder_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", research_system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "연구 내용에 기재된 수치나 그래프 등 시각화가 가능한 요소를 찾아, python 코드를 동작시켜 시각화를 해주세요."
            "최종적으로 시각화 한 사진을 다운로드 받을 수 있는 링크를 전달 해 주세요"
        ),
    ]
)
code_agent = create_react_agent(llm, tools=[python_repl_tool], prompt=coder_prompt)
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

* Writer Node 정의

In [ ]:
writer_system_prompt = (
    "당신은 고객이 보내준 연구 내용을 보고, 연구노트를 작성해 주는 에이전시의 작업자입니다."
    "Writer 역할을 담당하는 당신은, 고객의 연구 내용과 다른 작업의 작업 내용을 취합해 연구 노트 형식에 맞게 작성해야 합니다."
    "연구노트의 형식은 다음과 같습니다."
    "<연구 노트 형식>"
    "1. '-음', '-함' 과 같이 음슴체로 작성해 주세요."
    "2. 각 작업자와 고객이 보내준 연구 내용을 기반하여 작성해 주세요."
    "3. 마크다운 언어로 작성해 주세요."
    "4. 한국어로 작성해 주세요."
    ""
)

writer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", writer_system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "위 대화를 취합하여, 최종 연구노트를 작성해 주세요"
        ),
    ]
)

# LLM 정의
writer_llm = ChatOpenAI(model="gpt-4o")

def writer_agent(state):
    writer_chain = writer_prompt | writer_llm
    result = writer_chain.invoke(state)
    return {
        "messages": [writer_chain.invoke(state)]
    }


* Graph 정의 및 노드 추가

In [9]:
workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("Writer", writer_agent)
workflow.add_node("supervisor", supervisor_agent)

* Graph Edge 추가

In [10]:
# 모든 도구들은 다시 supervisor로 return 한다
for member in members:
    workflow.add_edge(member, "supervisor")

# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)

# 시작은 supervisor로
workflow.add_edge(START, "supervisor")

# 그래프 빌드
graph = workflow.compile()

# Run

In [11]:
research_content = """
모델이  학습되는  과정에서,  각  epoch 마다  정해진  metric 을  기록할  수  있도록 
시스템을  구현  하였음.  웹  상에서  UI 로  쉽게  확인하기  위해, WandB 를 
도입하였으며, DiceScore, DiceLoss 를  학습과  검증  과정에서  모두  기록해  보았음. 
CNN 계열의  Unet-3D 와  Vnet, Transformer 계열의  Unetr, Swin-Unetr  총  4 개의 
모델을  학습시켜본  결과는  아래와  같음 
Vnet 과  Swin-Unetr 의  모델이  Unet-3D 와  Unetr 에  비해  좋은  Dice Socre 를  가짐 
Vnet   
Dice Coefficient → 0.956 
Swin-Unetr   
Dice Coefficient → 0.956 
Unet-3D   
Dice Coefficient → 0.931 
Unetr   
Dice Coefficient → 0.891 
수치적인  성능으로  만  봤을  때는  Swin-Unet 과  Vnet 이  가장  좋은  성능을 
가졌지만,  이  두  모델에  대해서  시각화  하여  결과를  살펴  보았음. 
두  모델  모두  비슷하게  잘  예측하지만, Vent 은  가끔씩  예측영역이  잘게  쪼개지는 
현상이  나타났음.  이에  반해, SwinUnetr 은  전체적으로  시각화  품질  상태가 
양호해음.  때문에, SwinUnetr 을  다낭신  데이터에  한해  SOTA 모델이라  채택하여, 
해당  모델의  성능을  높여보는데  집중  하기로  함. 
"""

In [12]:
for s in graph.stream(
    {"messages": [HumanMessage(content=f"<연구 내용> {research_content}</연구 내용>")]},
    {"recursion_limit": 10},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='연구 결과에 따르면, Unet-3D, Vnet, Unetr, Swin-Unetr 네 가지 모델 중에서 Vnet과 Swin-Unetr가 가장 높은 Dice Coefficient를 기록했습니다. 두 모델 모두 0.956의 Dice Coefficient를 보였으며, 이는 Unet-3D(0.931)와 Unetr(0.891)에 비해 우수한 성능입니다.\n\n하지만 시각화 결과에서는 Vnet이 가끔 예측 영역이 잘게 쪼개지는 현상을 보였고, Swin-Unetr은 전체적으로 시각화 품질이 양호하여 더 나은 결과를 제공한 것으로 나타났습니다. 이러한 이유로, Swin-Unetr이 다낭신 데이터에 대한 SOTA(State-Of-The-Art) 모델로 채택되었으며, 연구팀은 Swin-Unetr의 성능을 더욱 향상시키는 데 집중하기로 결정했습니다.\n\nWandB를 활용하여 학습과 검증 과정에서 DiceScore와 DiceLoss를 기록하였고, 이를 통해 모델의 성능을 체계적으로 관리하고 분석할 수 있었습니다. 이 시스템이 모델 개발과 평가에 있어 유용한 도구로 작용했음을 알 수 있습니다.', additional_kwargs={}, response_metadata={}, name='Researcher')]}}
----
{'supervisor': {'next': 'Writer'}}
----
{'Writer': {'messages': [AIMessage(content='# 연구 노트\n\n1. 모델 학습 과정에서 각 epoch마다 정해진 metric을 기록할 수 있도록 시스템을 구현했음.  \n2. 웹 상에서 UI로 쉽게 확인하기 위해 WandB를 도입하였으며, DiceScore와 DiceLoss를 학습과 검증 과정에서 모두 기록해 보았음.  \n3. CNN 계열의 Unet-3D와 Vnet, Transformer 계